In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('urbansounds_features.csv')

In [37]:
# Replace the 'Label' column with the extracted value
df['Label'] = df['Label'].str.split('-').str[1]

# Save the DataFrame to a new CSV file if needed
df.to_csv('urbanfeatures_updated.csv', index=False)

# Read the updated CSV file into a new DataFrame
df_updated = pd.read_csv('urbanfeatures_updated.csv')

# Display the first few rows of the updated DataFrame
print(df_updated.head())

   chroma_stft  chroma_cqt  chroma_cens  melspectogram       rms     centroid  \
0     0.640295    0.745688     0.284223       1.256437  0.073593  1292.355561   
1     0.417256    0.601146     0.280195       3.291951  0.108715  2748.016622   
2     0.397634    0.561946     0.270126       2.221180  0.096742  1850.227016   
3     0.453350    0.540180     0.274063       0.034680  0.011608  1667.418114   
4     0.419926    0.583685     0.276122       2.758679  0.069048  2389.050124   

     bandwidth   contrast  flatness      rolloff  ...  mcffs_32  mcffs_33  \
0  1521.064554  21.011303  0.002814  2558.156978  ...  2.247787 -4.438743   
1  3053.747190  21.718229  0.018973  6838.907972  ... -0.566611 -4.938219   
2  2287.224210  19.870660  0.012228  4023.970658  ... -0.574458 -7.801946   
3  2069.784880  18.895311  0.009497  3313.623894  ... -2.907339 -4.239169   
4  2226.161005  20.386691  0.026576  4390.968631  ... -3.065629 -2.271313   

   mcffs_34  mcffs_35  mcffs_36  mcffs_37   mcffs_